In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

In [27]:
df_origin = pd.read_csv('./dataset.csv')

In [28]:
df_origin.head()

,text,raw_data_id
0,<|user|> What is high blood pressure? <|eos|> ...,0
1,<|user|> What are the risks of high blood pres...,0
2,<|user|> How is high blood pressure diagnosed?...,0
3,<|user|> What are the lifestyle changes that c...,0
4,<|user|> What are the medicines used to contro...,0


In [29]:
df = df_origin.drop(columns=['raw_data_id'], axis=1)

In [30]:
df.head()

,text
0,<|user|> What is high blood pressure? <|eos|> ...
1,<|user|> What are the risks of high blood pres...
2,<|user|> How is high blood pressure diagnosed?...
3,<|user|> What are the lifestyle changes that c...
4,<|user|> What are the medicines used to contro...


In [31]:
for i in range(5):
    print(f'{df.loc[i, "text"]}\n\n' + '*' * 50 + '\n')

<|user|> What is high blood pressure? <|eos|> <|ai|> High blood pressure is a condition where the force at which your heart pumps blood around your body is high. It is recorded with 2 numbers, the systolic pressure and the diastolic pressure, both measured in millimetres of mercury (mmHg).
References:
- https://www.nhs.uk/conditions/Blood-pressure-(high)/Pages/Introduction.aspx <|eos|> <|eod|>

**************************************************

<|user|> What are the risks of high blood pressure? <|eos|> <|ai|> Persistent high blood pressure can put extra strain on your blood vessels, heart, and other organs, such as the brain, kidneys, and eyes. It can increase your risk of serious and potentially life-threatening health conditions, such as heart disease, heart attacks, strokes, heart failure, peripheral arterial disease, aortic aneurysms, kidney disease, and vascular dementia.
References:
- https://www.nhs.uk/conditions/Blood-pressure-(high)/Pages/Introduction.aspx <|eos|> <|eod|>

*

In [35]:
df['text'] = df['text'].str.split('References:').str[0].str.strip() + ' <|eos|> <|eod|>'

In [37]:
for i in range(5):
    print(f'{df.loc[i, "text"]}\n\n' + '*' * 50 + '\n')

<|user|> What is high blood pressure? <|eos|> <|ai|> High blood pressure is a condition where the force at which your heart pumps blood around your body is high. It is recorded with 2 numbers, the systolic pressure and the diastolic pressure, both measured in millimetres of mercury (mmHg). <|eos|> <|eod|>

**************************************************

<|user|> What are the risks of high blood pressure? <|eos|> <|ai|> Persistent high blood pressure can put extra strain on your blood vessels, heart, and other organs, such as the brain, kidneys, and eyes. It can increase your risk of serious and potentially life-threatening health conditions, such as heart disease, heart attacks, strokes, heart failure, peripheral arterial disease, aortic aneurysms, kidney disease, and vascular dementia. <|eos|> <|eod|>

**************************************************

<|user|> How is high blood pressure diagnosed? <|eos|> <|ai|> The only way of knowing whether you have high blood pressure is to 